# 02 - Data from the Web


In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need! You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this brief tutorial to understand quickly how to use it.

In [393]:
from bs4 import BeautifulSoup
import requests
import json
import string
import pandas as pd
import re

## Data from QS ranking

Multiple strategies have been testes in order to find the data used to construct the table of the QS Rankings. However, only one of them led to the expected result. We opened the Web Inspector on the website. Among the ressources used by the page, a text file named 'XHR/357051.txt'  was containing the expected ranking with the desired informations. 

In [394]:
URL_QS = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508497791260'
r = requests.get(URL_QS)
page_body = r.json()
qs_df = pd.DataFrame(page_body['data'])

### Test to scrap additional data

In [396]:
URL_ADD = 'https://www.topuniversities.com/universities/stanford-university'
r = requests.get(URL_ADD)
page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser')
row = soup.find('div', {'class' : 'total faculty', 'class' : 'number'}).text
number = int(re.sub("[^\\d]",'',row)) #Je pense qu'il sera bien de regarder un peu pourquoi ce 'int' à rajouter. Si c'est unq uestion de version ou pas

In [397]:
number

4285

## Data from Times Higher Education ranking

In [451]:
URL_THE = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
r = requests.get(URL_THE)
page_body = r.json()
the_df = pd.DataFrame(page_body['data'])

In [452]:
type(the_df)

pandas.core.frame.DataFrame

### Clean the data

In [453]:
#we keep only the first 200

In [454]:
the_df = the_df[:200]

In [455]:
the_df;

## Quelles données nous sont inutiles? 

Je propose de garder: 
- name
- location
- nid (TBD)
- rank
- stat female: male (TBD)
- stats number students
- stats % intl students (to rename and maybe change the percentage by a number)
- stat student staff ratio


In [456]:
the_df = the_df.loc[ :, ['name', 'rank', 'location', 'nid', 'stats_female_male_ratio', 'stats_number_students', 'stats_pc_intl_students', 'stats_student_staff_ratio']]

In [457]:
the_df;

In [458]:
type(the_df)

pandas.core.frame.DataFrame

#### We calculate the number of International students

In [459]:
# Firts we need to remove the '%' symbol from the column "THE % International Students"
# Then we convert the string to int and we do the calculation

In [460]:
the_df['stats_number_students'] = pd.to_numeric(the_df['stats_number_students'].str.replace(',',''))
the_df['stats_pc_intl_students'] = pd.to_numeric(the_df['stats_pc_intl_students'].str.replace('%',''))
for index, row in the_df.iterrows():
    the_df.loc[index, 'stats_pc_intl_students'] = round(row['stats_number_students']*row['stats_pc_intl_students']*0.01)


#### We rename the Data in order to combine the two DF later

In [462]:
the_df = the_df.rename(index=str, columns={"rank": "THE rank", "nid": "THE nid",
                                  "stats_female_male_ratio" : "THE Ration Male/Female",
                                  "stats_number_students" : "THE Total Students",
                                  "stats_pc_intl_students" : "THE International Students",
                                  "stats_student_staff_ratio" : "THE Ratio Students/Staff"
                                 });

In [466]:
the_df

,name,THE rank,location,THE nid,THE Ration Male/Female,THE Total Students,THE International Students,THE Ratio Students/Staff
0,University of Oxford,1,United Kingdom,468,46 : 54,20409,7755,11.2
1,University of Cambridge,2,United Kingdom,470,45 : 55,18389,6436,10.9
2,California Institute of Technology,=3,United States,128779,31 : 69,2209,596,6.5
3,Stanford University,=3,United States,467,42 : 58,15845,3486,7.5
4,Massachusetts Institute of Technology,5,United States,471,37 : 63,11177,3800,8.7
5,Harvard University,6,United States,466,None,20326,5285,8.9
6,Princeton University,7,United States,469,45 : 55,7955,1909,8.3
7,Imperial College London,8,United Kingdom,472,37 : 63,15857,8721,11.4
8,University of Chicago,9,United States,473,44 : 56,13525,3381,6.2
9,ETH Zurich – Swiss Federal Institute of Techno...,=10,Switzerland,479,31 : 69,19233,7309,14.6
